In [7]:
# 1-) Yeni Ürünlerin sektörlerini bulmak

import pandas as pd

# CSV dosyalarını yükleyelim
df_2002 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS02_V202401\product_codes_HS02_V202401.csv")
df_2007 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS07_V202401\product_codes_HS07_V202401.csv")
df_2012 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS12_V202401\product_codes_HS12_V202401.csv")
df_2017 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS17_V202401\product_codes_HS17_V202401.csv")
df_2022 = pd.read_csv(r"C:\Users\oguzo\Desktop\BACI\BACI_HS22_V202401\product_codes_HS22_V202401.csv")

# 2002, 2007, 2012, 2017 ve 2022 yıllarına ait product code'ları set yap
products_2002 = set(df_2002['code'])
products_2007 = set(df_2007['code'])
products_2012 = set(df_2012['code'])
products_2017 = set(df_2017['code'])
products_2022 = set(df_2022['code'])

# 2002, 2012, 2017 ve 2022'de yeni eklenen ürünler
new_products_2007 = products_2007 - products_2002
new_products_2007_df = df_2007[df_2007['code'].isin(new_products_2007)].copy()
new_products_2012 = products_2012 - products_2007
new_products_2012_df = df_2012[df_2012['code'].isin(new_products_2012)].copy()
new_products_2017 = products_2017 - products_2012
new_products_2017_df = df_2017[df_2017['code'].isin(new_products_2017)].copy()
new_products_2022 = products_2022 - products_2017
new_products_2022_df = df_2022[df_2022['code'].isin(new_products_2022)].copy()

# Her yıla ait veri setine "Yıl" sütunu ekleyelim
new_products_2007_df['Yıl'] = 2007
new_products_2012_df['Yıl'] = 2012
new_products_2017_df['Yıl'] = 2017
new_products_2022_df['Yıl'] = 2022

# Üç yıldaki yeni ürünleri aynı data frame içine al
combined_new_products = pd.concat([new_products_2022_df, new_products_2017_df, new_products_2012_df, new_products_2007_df], ignore_index=True)

# 6 basamaklı HS kodlarını 10000'e bölerek 2 basamaklı sektörlerini bul
combined_new_products['2 Basamaklı HS Kodu'] = combined_new_products['code'] // 10000

# Atlas verisini çağırıp 2 basamaklı HS kodlarının isimlerini çekelim
combined_atlas_df = pd.read_excel('0-) Atlastan Çekilmiş HS92 İsimleri 1 Basamak İçin.xlsx')
combined_atlas_df['2 Basamaklı HS Kodları'] = combined_atlas_df['2 Basamaklı HS Kodları'].str.extract(r'\((\d{2})')
combined_atlas_df['1 Basamaklı HS Kodları'] = combined_atlas_df['1 Basamaklı HS Kodları'].str.extract(r'\((\d{1})')

# Bir sözlük oluşturarak 2 basamaklı HS kodları ve isimleri arasında bir map oluşturalım
code_2_to_name_mapping = dict(zip(combined_atlas_df['2 Basamaklı HS Kodları'], combined_atlas_df['2 Basamaklı HS İsimleri']))
code_2_to_code_1_mapping = dict(zip(combined_atlas_df['2 Basamaklı HS Kodları'], combined_atlas_df['1 Basamaklı HS Kodları']))
code_1_to_name_mapping = dict(zip(combined_atlas_df['1 Basamaklı HS Kodları'], combined_atlas_df['1 Basamaklı HS İsimleri']))

# 2 basamaklı HS kodlarını ve bunlara karşılık gelen 1 basamaklı kodları ve isimleri ekleyelim
combined_new_products['2 Basamaklı HS İsmi'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2).map(code_2_to_name_mapping)
combined_new_products['1 Basamaklı HS Kodu'] = combined_new_products['2 Basamaklı HS Kodu'].astype(str).str.zfill(2).map(code_2_to_code_1_mapping)
combined_new_products['1 Basamaklı HS İsmi'] = combined_new_products['1 Basamaklı HS Kodu'].map(code_1_to_name_mapping)

# 6 basamaklı HS kodunu ve detaylı sektör ismini zaten koruyoruz
combined_new_products = combined_new_products.rename(columns={'description': 'Detaylı Sektör İsmi', 'code': '6 Basamaklı HS Kodu'})  

# "Yıl" sütununu en başa taşıyalım
cols = ['Yıl'] + [col for col in combined_new_products.columns if col != 'Yıl']
combined_new_products = combined_new_products[cols]

# Excele kaydet
combined_new_products.to_excel("1-) Yeni ürünlerin sektörleri.xlsx")

In [2]:
import pandas as pd

# Dosyayı yükle
file_path = '1-) 2012, 2017 ve 2022\'deki yeni ürünlerin sektörleri.xlsx'
new_products_df = pd.read_excel(file_path)

# Grup bazında sayıları hesapla: Yıl ve 1 Basamaklı HS Koduna göre
sector_grouping_by_year = new_products_df.groupby(['Yıl', '1 Basamaklı HS Kodu', '1 Basamaklı HS İsmi']).size().reset_index(name='Sayı')

# Toplam ürün sayısını yıl bazında hesapla
total_by_year = sector_grouping_by_year.groupby('Yıl')['Sayı'].sum().reset_index(name='Toplam Ürün Sayısı')

# Her yıl için 1 basamaklı HS kodlarının oranlarını hesapla
sector_grouping_by_year = sector_grouping_by_year.merge(total_by_year, on='Yıl')
sector_grouping_by_year['Oran (%)'] = (sector_grouping_by_year['Sayı'] / sector_grouping_by_year['Toplam Ürün Sayısı']) * 100

# Sonuçları Excel dosyasına kaydet
output_file_path = '1_basamakli_HS_kodlarina_gore_oranlar.xlsx'
sector_grouping_by_year.to_excel(output_file_path, index=False)

print(f"Dosya başarıyla kaydedildi: {output_file_path}")


Dosya başarıyla kaydedildi: 1_basamakli_HS_kodlarina_gore_oranlar.xlsx


In [3]:
import pandas as pd

# Dış ticaret verisi için temel değişkenler
# t: year, i: exporter, j: importer, k: product, v: value, q: quantity

# Dış ticaret verilerini çekme fonksiyonu (veri kaynaklarını kendiniz ayarlamalısınız)
def get_trade_data():
    # Bu kısımda dış ticaret verisini kaynağınıza bağlı olarak API'den çekebilir ya da dosya okuyabilirsiniz.
    # Örnek veriyi dış bir kaynaktan okuyabiliriz.
    trade_data = pd.DataFrame({
        't': [2012, 2012, 2017, 2017, 2022, 2022],
        'i': ['Country_A', 'Country_B', 'Country_A', 'Country_B', 'Country_A', 'Country_B'],
        'j': ['Country_X', 'Country_Y', 'Country_X', 'Country_Y', 'Country_X', 'Country_Y'],
        'k': [1, 1, 2, 2, 3, 3],  # 1 basamaklı HS kodu
        'v': [1000, 1500, 2000, 2500, 3000, 3500]  # Trade value
    })
    return trade_data

# Dış ticaret verisini çek
trade_data = get_trade_data()

# 1 basamaklı HS kodlarına göre gruplama ve toplam değer hesaplama
trade_grouping_by_year = trade_data.groupby(['t', 'k']).agg({'v': 'sum'}).reset_index()
trade_grouping_by_year.columns = ['Yıl', '1 Basamaklı HS Kodu', 'Toplam Değer (Value)']

# Her yıl için toplam dış ticaret değerini hesapla
total_trade_by_year = trade_grouping_by_year.groupby('Yıl')['Toplam Değer (Value)'].sum().reset_index(name='Yıl Toplam Değer')

# Yıl bazında oranları hesapla
trade_grouping_by_year = trade_grouping_by_year.merge(total_trade_by_year, on='Yıl')
trade_grouping_by_year['Oran (%)'] = (trade_grouping_by_year['Toplam Değer (Value)'] / trade_grouping_by_year['Yıl Toplam Değer']) * 100

# Sonuçları Excel dosyasına kaydet
output_file_path = 'dis_ticaret_oranlari.xlsx'
trade_grouping_by_year.to_excel(output_file_path, index=False)

print(f"Dosya başarıyla kaydedildi: {output_file_path}")


Dosya başarıyla kaydedildi: dis_ticaret_oranlari.xlsx
